In [1]:
import numpy as np
import tensorflow as tf
import mdn

In [2]:
import tensorflow.keras.backend as K

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, InputLayer, Attention
from tensorflow.keras.layers import (Conv2D, Input, Reshape, 
                                     Lambda, Dense, Conv2DTranspose)

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
import matplotlib.pyplot as plt
import os
import utils

from tensorflow.keras.callbacks import (EarlyStopping, ModelCheckpoint, 
                                       TensorBoard, Callback)
import datetime
from time import time
from utils import TrainTimeCallback

In [4]:
import tensorflow_probability as tfp
tfd = tfp.distributions

In [5]:
import matplotlib.pyplot as plt

In [6]:
seq_len = 128
act_len = 3
n_mixtures = 5
output_dims = 32

In [7]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:          16020        1703        7044           3        7273       13946
Swap:          4095          43        4051


In [8]:
num_instances = len(os.listdir('./data/states'))

In [9]:
def load_folder(path):
    files = os.listdir(path)
    _1 = np.load(os.path.join(path, files[0]))
    data = np.zeros((len(files), *_1.shape))
    for i, fname in enumerate(files):
        data[i] = np.load(os.path.join(path, fname))
    return data

In [10]:
z_states = load_folder('./data/z_states')

In [11]:
z_states.shape

(154624, 32)

In [12]:
actions = load_folder('./data/actions')

In [13]:
actions.shape

(154624, 3)

In [14]:
154624 / 128 / 128

9.4375

In [15]:
pair = np.concatenate((z_states, actions), axis=1)

In [16]:
pair.shape

(154624, 35)

In [17]:
latent_dataset = tf.data.Dataset.from_tensor_slices(pair)

In [18]:
sequences = latent_dataset.batch(seq_len + 1, drop_remainder=True)

In [19]:
sequences

<BatchDataset shapes: (129, 35), types: tf.float64>

In [20]:
def split_input_target(chunk):
    input_z = chunk[:-1]
    target_z = chunk[1:, :32]
    return input_z, target_z

In [21]:
dataset = sequences.map(split_input_target)

In [22]:
dataset

<MapDataset shapes: ((128, 35), (128, 32)), types: (tf.float64, tf.float64)>

In [23]:
dataset = dataset.shuffle(10000).batch(utils.BATCH_SIZE, drop_remainder=True)

In [24]:
dataset

<BatchDataset shapes: ((128, 128, 35), (128, 128, 32)), types: (tf.float64, tf.float64)>

In [25]:
i = 0
for a, b in dataset:
    i += 1

In [26]:
i

9

In [27]:
percent_20 = i // 5
val = dataset.take(percent_20)
train = dataset.skip(percent_20)

In [28]:
val

<TakeDataset shapes: ((128, 128, 35), (128, 128, 32)), types: (tf.float64, tf.float64)>

In [29]:
train

<SkipDataset shapes: ((128, 128, 35), (128, 128, 32)), types: (tf.float64, tf.float64)>

In [35]:
!mkdir "./logs/"
!mkdir "./logs/fit"
!rm "./logs/fit/*"

mkdir: cannot create directory ‘./logs/’: File exists
mkdir: cannot create directory ‘./logs/fit’: File exists
rm: cannot remove './logs/fit/*': No such file or directory


In [36]:
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [37]:
seq_len = 128
act_len = 3

In [38]:
callbacks = [
    TrainTimeCallback(),
    ModelCheckpoint('./best_mdn_rnn.h5', save_best_only=True, 
                    monitor='val_loss', save_weights_only=True),
    TensorBoard(log_dir=log_dir, histogram_freq=1)
]

In [41]:
output_dims = 32
n_mixes     = 5
M = Sequential([
    Input((seq_len, act_len + utils.LATENT_SIZE)),
    LSTM(256, return_sequences=True),
    mdn.MDN(output_dims, n_mixes)
])

M.compile(loss=mdn.get_mixture_loss_func(output_dims, n_mixes), 
          optimizer=tf.keras.optimizers.Adam(), 
          callbacks=callbacks,
         )

M.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128, 256)          299008    
_________________________________________________________________
mdn (MDN)                    (None, 128, 325)          83525     
Total params: 382,533
Trainable params: 382,533
Non-trainable params: 0
_________________________________________________________________


In [42]:
history = M.fit(train, epochs=100, validation_data=val)

Epoch 1/100
8/8 [==============================] - 3s 372ms/step - loss: 38.5307 - val_loss: 0.0000e+00
Epoch 2/100
8/8 [==============================] - 0s 59ms/step - loss: 33.1567 - val_loss: 30.0616
Epoch 3/100
8/8 [==============================] - 0s 57ms/step - loss: 28.7403 - val_loss: 26.8452
Epoch 4/100
8/8 [==============================] - 0s 56ms/step - loss: 26.0379 - val_loss: 23.8789
Epoch 5/100
8/8 [==============================] - 0s 59ms/step - loss: 23.8192 - val_loss: 23.7365
Epoch 6/100
8/8 [==============================] - 0s 56ms/step - loss: 22.7239 - val_loss: 21.4102
Epoch 7/100
8/8 [==============================] - 0s 55ms/step - loss: 21.9298 - val_loss: 21.5287
Epoch 8/100
8/8 [==============================] - 0s 55ms/step - loss: 20.8378 - val_loss: 20.9831
Epoch 9/100
8/8 [==============================] - 0s 57ms/step - loss: 20.8103 - val_loss: 19.6289
Epoch 10/100
8/8 [==============================] - 0s 54ms/step - loss: 20.5868 - val_loss: 19.

8/8 [==============================] - 0s 56ms/step - loss: 15.2063 - val_loss: 14.2729
Epoch 83/100
8/8 [==============================] - 0s 57ms/step - loss: 14.4362 - val_loss: 14.1446
Epoch 84/100
8/8 [==============================] - 0s 53ms/step - loss: 14.1464 - val_loss: 14.7200
Epoch 85/100
8/8 [==============================] - 0s 57ms/step - loss: 14.6522 - val_loss: 14.2643
Epoch 86/100
8/8 [==============================] - 0s 54ms/step - loss: 14.2669 - val_loss: 13.7082
Epoch 87/100
8/8 [==============================] - 0s 57ms/step - loss: 13.9680 - val_loss: 14.0284
Epoch 88/100
8/8 [==============================] - 0s 58ms/step - loss: 14.1267 - val_loss: 15.0762
Epoch 89/100
8/8 [==============================] - 0s 58ms/step - loss: 14.4318 - val_loss: 13.9782
Epoch 90/100
8/8 [==============================] - 0s 53ms/step - loss: 13.8827 - val_loss: 13.6977
Epoch 91/100
8/8 [==============================] - 0s 55ms/step - loss: 13.7656 - val_loss: 13.7779
Epo

In [ ]:
plt.plot(history.history['loss'])

In [ ]:
plt.plot(history.history['val_loss'])